In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [2]:
data_backup = data.iloc[3524:]
data = data.iloc[:3524]

### model5 for different time steps:

#### 10 bar lag

In [3]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high = seasonal_decompose(data['high'], model='additive', period=30)
trend_high = decomposition_high.trend

decomposition_low = seasonal_decompose(data['low'], model='additive', period=30)
trend_low = decomposition_low.trend

In [5]:
data['trend_high'] = trend_high
data['trend_low'] = trend_low

In [6]:
import pandas as pd

for i in range(1,11):
    data[f'trend_h+{i}'] = data['trend_high'].shift(i)
for i in range(1,11):
    data[f'trend_l+{i}'] = data['trend_low'].shift(i)
data.dropna(axis=0, inplace=True)

In [7]:
inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17]]
target_high = data.iloc[:, [6]]

inputs_low = data.iloc[:, [18,19,20,21,22,23,24,25,26,27]]
target_low = data.iloc[:, [7]]

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_high_10 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high_10.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low_10 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low_10.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

#### data reading:

In [10]:
data_backup = pd.read_csv('data/^NDX_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [11]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high_backup = seasonal_decompose(data_backup['high'], model='additive', period=30)
trend_high_backup = decomposition_high_backup.trend

decomposition_low_backup = seasonal_decompose(data_backup['low'], model='additive', period=30)
trend_low_backup = decomposition_low_backup.trend

In [12]:
data_backup['trend_high'] = trend_high_backup
data_backup['trend_low'] = trend_low_backup

In [13]:
import pandas as pd

for i in range(1,11):
    data_backup[f'trend_h+{i}'] = data_backup['trend_high'].shift(i)
for i in range(1,11):
    data_backup[f'trend_l+{i}'] = data_backup['trend_low'].shift(i)
data_backup.dropna(axis=0, inplace=True)

In [14]:
inputs_high_backup = data_backup.iloc[:, [8,9,10,11,12,13,14,15,16,17]]
inputs_low_backup = data_backup.iloc[:, [18,19,20,21,22,23,24,25,26,27]]

In [15]:
df_pred = dict()

scaler_input = MinMaxScaler()
data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_high']])

y_pred_h_backup = random_tree_high_10.predict(data_backup_h_scaled)
y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

scaler_input = MinMaxScaler()
data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_low']])

y_pred_l_backup = random_tree_low_10.predict(data_backup_l_scaled)
y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

df_pred = {'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}

In [16]:
data_backup.reset_index(inplace=True)

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

score_h = mean_absolute_error(data_backup[['high']], df_pred['pred_h'])
r2_h = r2_score(data_backup[['high']], df_pred['pred_h'])
data_backup['mae_h'] = score_h

score_l = mean_absolute_error(data_backup[['low']], df_pred['pred_l'])
r2_l = r2_score(data_backup[['low']], df_pred['pred_l'])
data_backup['mae_l'] = score_l

print(f'METRICS FOR NASDAQ INDEX:')
print(f'score high: {score_h}')
print(f'score low: {score_l}')

print(f'- R2 METRICS FOR NASDAQ INDEX:')
print(f'r2 high: {r2_h}')
print(f'r2 low: {r2_l}')

METRICS FOR NASDAQ INDEX:
score high: 81.8052708329642
score low: 89.86792941464195
- R2 METRICS FOR NASDAQ INDEX:
r2 high: 0.9987990885885053
r2 low: 0.9984928620114331


In [17]:
remaining_columns = ('date','open','high','low','close','volume','trend_high','trend_low','mae_h','mae_l')

data_backup = data_backup[list(remaining_columns)]

y_pred_h_series = pd.Series(df_pred['pred_h'].reshape(1,-1)[0])
y_pred_l_series = pd.Series(df_pred['pred_l'].reshape(1,-1)[0])
data_backup['pred_high'] = y_pred_h_series
data_backup['pred_low'] = y_pred_l_series

data_backup.to_excel(f'data_w_pred_m5/index_w_pred_m5_10bar.xlsx')

/tmp/ipykernel_148943/2133779364.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_high'] = y_pred_h_series
/tmp/ipykernel_148943/2133779364.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_low'] = y_pred_l_series


#### 30 bar lag:

In [18]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

data_backup = data.iloc[3524:]
data = data.iloc[:3524]

from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high = seasonal_decompose(data['high'], model='additive', period=30)
trend_high = decomposition_high.trend

decomposition_low = seasonal_decompose(data['low'], model='additive', period=30)
trend_low = decomposition_low.trend

data['trend_high'] = trend_high
data['trend_low'] = trend_low

import pandas as pd

for i in range(1,31):
    data[f'trend_h+{i}'] = data['trend_high'].shift(i)
for i in range(1,31):
    data[f'trend_l+{i}'] = data['trend_low'].shift(i)
data.dropna(axis=0, inplace=True)

In [19]:
inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37]]
target_high = data.iloc[:, [6]]

inputs_low = data.iloc[:, [38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67]]
target_low = data.iloc[:, [7]]

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_high_30 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high_30.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low_30 = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low_30.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

#### data reading:

In [21]:
data_backup = pd.read_csv('data/^NDX_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high_backup = seasonal_decompose(data_backup['high'], model='additive', period=30)
trend_high_backup = decomposition_high_backup.trend

decomposition_low_backup = seasonal_decompose(data_backup['low'], model='additive', period=30)
trend_low_backup = decomposition_low_backup.trend

data_backup['trend_high'] = trend_high_backup
data_backup['trend_low'] = trend_low_backup

import pandas as pd

for i in range(1,31):
    data_backup[f'trend_h+{i}'] = data_backup['trend_high'].shift(i)
for i in range(1,31):
    data_backup[f'trend_l+{i}'] = data_backup['trend_low'].shift(i)
data_backup.dropna(axis=0, inplace=True)

In [22]:
inputs_high_backup = data_backup.iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37]]
inputs_low_backup = data_backup.iloc[:, [38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67]]

In [23]:
df_pred = dict()

scaler_input = MinMaxScaler()
data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_high']])

y_pred_h_backup = random_tree_high_30.predict(data_backup_h_scaled)
y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

scaler_input = MinMaxScaler()
data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_low']])

y_pred_l_backup = random_tree_low_30.predict(data_backup_l_scaled)
y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

df_pred = {'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}

In [24]:
data_backup.reset_index(inplace=True)

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

score_h = mean_absolute_error(data_backup[['high']], df_pred['pred_h'])
r2_h = r2_score(data_backup[['high']], df_pred['pred_h'])
data_backup['mae_h'] = score_h

score_l = mean_absolute_error(data_backup[['low']], df_pred['pred_l'])
r2_l = r2_score(data_backup[['low']], df_pred['pred_l'])
data_backup['mae_l'] = score_l

print(f'METRICS FOR NASDAQ INDEX:')
print(f'score high: {score_h}')
print(f'score low: {score_l}')

print(f'- R2 METRICS FOR NASDAQ INDEX:')
print(f'r2 high: {r2_h}')
print(f'r2 low: {r2_l}')

METRICS FOR NASDAQ INDEX:
score high: 99.25664050968639
score low: 105.15383787328467
- R2 METRICS FOR NASDAQ INDEX:
r2 high: 0.997961661178575
r2 low: 0.9976528829386956


In [25]:
remaining_columns = ('date','open','high','low','close','volume','trend_high','trend_low','mae_h','mae_l')

data_backup = data_backup[list(remaining_columns)]

y_pred_h_series = pd.Series(df_pred['pred_h'].reshape(1,-1)[0])
y_pred_l_series = pd.Series(df_pred['pred_l'].reshape(1,-1)[0])
data_backup['pred_high'] = y_pred_h_series
data_backup['pred_low'] = y_pred_l_series

data_backup.to_excel(f'data_w_pred_m5/index_w_pred_m5_30bar.xlsx')